In [1]:
print("SHREE RAM")

SHREE RAM


In [2]:
#!pip install anvil-uplink

In [3]:
pip install -qqq sentence-transformers imbalanced-learn pandas scikit-learn numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00


In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Sentence Embeddings (BGE-Large) with Random Forest Classification

This script uses BGE-Large sentence embeddings with a Random Forest classifier
for sentiment analysis and sarcasm detection.
"""

import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE


def download_datasets():
    """Download required datasets if they don't exist."""
    datasets = [
        ("https://gist.githubusercontent.com/Phani-2608/1610e06ac549fc3a17c1027eeb9275c1/raw/f07273b926b4a8ff45394b36bfa2b9e969098a1a/consolidated.csv", "consolidated csv")
    ]

    for url, filename in datasets:
        if not os.path.exists(filename):
            print(f"Downloading {filename}...")
            os.system(f"wget {url}")


def load_data():
    df = pd.read_csv("consolidated.csv")
    return df


def get_bert_features(text, tokenizer, model):
    """Extract embeddings using SentenceTransformer model."""
    embedding = model.encode(text, convert_to_numpy=True)
    return embedding


def process_data(df, tokenizer, model):
    """Process data to extract embeddings."""
    print("Extracting sentence embeddings...")
    embeddings = []

    for i, text in enumerate(df['sentence']):
        if i % 100 == 0:
            print(f"Processing {i}/{len(df)} sentences...")
        embedding = get_bert_features(text, tokenizer, model)
        embeddings.append(embedding)

    return np.vstack(embeddings)


def apply_smote(X, y):
    """Apply SMOTE to handle class imbalance."""
    print("Applying SMOTE to balance classes...")
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    return X_resampled, y_resampled


def train_and_evaluate_model(X_train, X_test, y_train, y_test):
    """Train Random Forest model and evaluate its performance."""
    print("Training Random Forest classifier...")
    clf = RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1,
        verbose=1
    )

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:\n", report)

    return clf, accuracy, report


def main():
    """Main function to orchestrate the entire process."""
    print("Loading BGE-Large sentence embedding model...")
    model = SentenceTransformer('BAAI/bge-large-en-v1.5')
    tokenizer = None  # Not used with SentenceTransformer

    download_datasets()

    df = load_data()
    print(f"Dataset shape: {df.shape}")
    print(f"Class distribution:\n{df['label'].value_counts()}")

    X = process_data(df, tokenizer, model)
    y = df['label'].values

    X_resampled, y_resampled = apply_smote(X, y)
    print(f"Shape after SMOTE: X={X_resampled.shape}, y={y_resampled.shape}")

    X_train, X_test, y_train, y_test = train_test_split(
        X_resampled, y_resampled, test_size=0.2, random_state=42
    )

    model, accuracy, report = train_and_evaluate_model(X_train, X_test, y_train, y_test)

    print("Process completed successfully!")

    return model


if __name__ == "__main__":
    model = main()


Loading BGE-Large sentence embedding model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Dataset shape: (55328, 2)
Class distribution:
label
0    29970
1    25358
Name: count, dtype: int64
Extracting sentence embeddings...
Processing 0/55328 sentences...
Processing 100/55328 sentences...
Processing 200/55328 sentences...
Processing 300/55328 sentences...
Processing 400/55328 sentences...
Processing 500/55328 sentences...
Processing 600/55328 sentences...
Processing 700/55328 sentences...
Processing 800/55328 sentences...
Processing 900/55328 sentences...
Processing 1000/55328 sentences...
Processing 1100/55328 sentences...
Processing 1200/55328 sentences...
Processing 1300/55328 sentences...
Processing 1400/55328 sentences...
Processing 1500/55328 sentences...
Processing 1600/55328 sentences...
Processing 1700/55328 sentences...
Processing 1800/55328 sentences...
Processing 1900/55328 sentences...
Processing 2000/55328 sentences...
Processing 2100/55328 sentences...
Processing 2200/55328 sentences...
Processing 2300/55328 sentences...
Processing 2400/55328 sentences...
Pro

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.7min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s


Accuracy: 0.9597
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.97      0.96      5998
           1       0.97      0.95      0.96      5990

    accuracy                           0.96     11988
   macro avg       0.96      0.96      0.96     11988
weighted avg       0.96      0.96      0.96     11988

Process completed successfully!


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.3s finished


In [6]:
def predict_sarcasm(query):

  query_emb = get_bert_features(query, None, SentenceTransformer('BAAI/bge-large-en-v1.5'))
  res = model.predict(query_emb.reshape(1,-1))[0]
  if res:
    return 'The query is sarcastic'
  else:
    return 'the query is not sarcastic'

query = 'eating in a tall building helps grow taller'
predict_sarcasm(query)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


'The query is sarcastic'